https://gist.github.com/superkojiman/10927867
getting artist genre code, if needed

This code was creating merged dataset with sentiment scores. It doesn't need to be re run bc it takes a really long time to run.

In [1]:
import re
import numpy as np
import pandas as pd
import csv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
movie_meta = pd.read_csv('models/MovieSummaries/movie.metadata.tsv', header=None, delimiter='\t',encoding='utf-8',names=['WikiID', 'FreebaseID', 'Title', 'ReleaseDate', 'Revenue', 'Runtime', 'Languages', 'Countries', 'Genres'])
movie_meta = movie_meta[['Title','WikiID','Genres']]
movie_meta.head()

,Title,WikiID,Genres
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,Getting Away with Murder: The JonBenét Ramsey ...,3196793,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,Brun bitter,28463795,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}"


In [3]:
plots = pd.read_csv('models/MovieSummaries/plot_summaries.txt', sep='\t',names=['WikiID', 'Summary'])
plots.head()

,WikiID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [4]:
merged = pd.merge(movie_meta, plots, on='WikiID')
print(len(merged))
merged.set_index('Title')
merged.head()
#print(merged['Summary'][0])

42204


,Title,WikiID,Genres,Summary
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th..."
1,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...
2,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra..."
3,The Sorcerer's Apprentice,18998739,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns..."
4,Little city,6631279,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a..."


In [5]:
analyzer = SentimentIntensityAnalyzer()

In [6]:
merged['pos'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['pos'], axis=1)
merged['neg'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['neg'], axis=1)

In [7]:
merged['compound'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['compound'], axis=1)

In [8]:
merged['neu'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['neu'], axis=1)

In [13]:
genres = []
for x in merged['Genres']:
    g = re.findall(': \"(.*?)\"', x)
    genres.append(g)

In [14]:
merged['Genres'] = genres
merged.head()

,Title,WikiID,Genres,Summary,pos,neg,compound,neu
0,Ghosts of Mars,975900,"[Thriller, Science Fiction, Horror, Adventure,...","Set in the second half of the 22nd century, th...",0.065,0.150,-0.9913,0.786
1,White Of The Eye,9363483,"[Thriller, Erotic thriller, Psychological thri...",A series of murders of rich young women throug...,0.078,0.198,-0.9985,0.724
2,A Woman in Flames,261236,[Drama],"Eva, an upper class housewife, becomes frustra...",0.100,0.065,0.9604,0.835
3,The Sorcerer's Apprentice,18998739,"[Family Film, Fantasy, Adventure, World cinema]","Every hundred years, the evil Morgana returns...",0.092,0.131,-0.8885,0.776
4,Little city,6631279,"[Romantic comedy, Ensemble Film, Comedy-drama,...","Adam, a San Francisco-based artist who works a...",0.089,0.108,-0.7097,0.804


In [15]:
merged.to_csv (r'merged_data.csv', index = False, header=True)